# Importação de LOGs para MySQL e geeração de relatórios

### Primeiramente vamos importar as bibliotecas necessárias. Após este passo vamos ler o arquivo e criar um pandas dataframe com ele

In [1]:
# Importar as bibliotecas necessárias
import pandas as pd
import json
import mysql.connector
from mysql.connector import errorcode

In [2]:
#Define meu arquivo de log
file = 'logs.txt'

In [3]:
#Le o arquivo e salva na variavel Log
log = []
for line in open(file, 'r'):
    log.append(json.loads(line))

In [4]:
df = pd.DataFrame(log)

In [5]:
#Cria um id para o log baseando no indice
df['id'] = df.index+1

#Exibe as 5 primeiras linhas
df.head()

,request,upstream_uri,response,authenticated_entity,route,service,latencies,client_ip,started_at,id
0,"{'method': 'GET', 'uri': '/', 'url': 'http://y...",/,"{'status': 500, 'size': 878, 'headers': {'Cont...",{'consumer_id': {'uuid': '72b34d31-4c14-3bae-9...,"{'created_at': 1564823899, 'hosts': 'miller.co...","{'connect_timeout': 60000, 'created_at': 15635...","{'proxy': 1836, 'kong': 8, 'request': 1058}",75.241.168.121,1566660387,1
1,"{'method': 'GET', 'uri': '/', 'url': 'http://l...",/,"{'status': 204, 'size': 313, 'headers': {'Cont...",{'consumer_id': {'uuid': 'f643db14-a82d-30c2-8...,"{'created_at': 1561154217, 'hosts': 'schneider...","{'connect_timeout': 60000, 'created_at': 15473...","{'proxy': 1727, 'kong': 10, 'request': 2185}",88.248.178.118,1556832648,2
2,"{'method': 'GET', 'uri': '/', 'url': 'http://w...",/,"{'status': 500, 'size': 366, 'headers': {'Cont...",{'consumer_id': {'uuid': 'beceaa24-823b-3bf9-9...,"{'created_at': 1545521760, 'hosts': 'hilpert.c...","{'connect_timeout': 60000, 'created_at': 15495...","{'proxy': 896, 'kong': 9, 'request': 2129}",56.54.196.221,1543186574,3
3,"{'method': 'GET', 'uri': '/', 'url': 'http://v...",/,"{'status': 401, 'size': 522, 'headers': {'Cont...",{'consumer_id': {'uuid': '7ba24e1f-ed19-31b2-a...,"{'created_at': 1543036091, 'hosts': 'corkery.o...","{'connect_timeout': 60000, 'created_at': 15444...","{'proxy': 1209, 'kong': 12, 'request': 1274}",164.83.216.199,1544521945,4
4,"{'method': 'GET', 'uri': '/', 'url': 'http://l...",/,"{'status': 401, 'size': 963, 'headers': {'Cont...",{'consumer_id': {'uuid': '1696f3c4-7732-38f2-9...,"{'created_at': 1543036091, 'hosts': 'corkery.o...","{'connect_timeout': 60000, 'created_at': 15444...","{'proxy': 1708, 'kong': 20, 'request': 2244}",154.78.181.43,1538597571,5


### Após o dataframe criado vamos criar novos dataframes para separar os dados dos logs

##### Request, Response, Route, Service e Latencies

In [6]:
# Cria um novo Dataframe com os Requests
df_request = (
    df["request"]
    .apply(pd.Series)
    .merge(df['id'], left_index=True, right_index = True)
)

# Elimina a coluna do Dataframe principal
df.drop(['request'], axis='columns', inplace=True)

# Extrair os dados do header
df_request = (
    df_request["headers"]
    .apply(pd.Series)
    .merge(df_request, left_index=True, right_index = True)
)

#Elimina a coluna 
df_request.drop(['headers'], axis='columns', inplace=True)

# Renomeia a coluna
df_request = df_request.rename(columns = {'id': 'log_id'}, inplace = False)

# Transforma a coluna com lista em string
df_request['querystring'] = [','.join(map(str, l)) for l in df_request['querystring']]

# Exibe os 5 primeiros registros
df_request.head()

,accept,host,user-agent,method,uri,url,size,querystring,log_id
0,*/*,yost.com,curl/7.37.1,GET,/,http://yost.com,174,,1
1,*/*,labadie.com,curl/7.37.1,GET,/,http://labadie.com,159,,2
2,*/*,wisozk.biz,curl/7.37.1,GET,/,http://wisozk.biz,79,,3
3,*/*,veum.org,curl/7.37.1,GET,/,http://veum.org,62,,4
4,*/*,ledner.com,curl/7.37.1,GET,/,http://ledner.com,167,,5


In [7]:
# Cria um novo Dataframe com o Response
df_response = (
    df["response"]
    .apply(pd.Series)
    .merge(df['id'], left_index=True, right_index = True)
)

# Elimina a coluna do Dataframe principal
df.drop(['response'], axis='columns', inplace=True)

# Extrair os dados do header
df_response = (
    df_response["headers"]
    .apply(pd.Series)
    .merge(df_response, left_index=True, right_index = True)
)

# Elimina a coluna 
df_response.drop(['headers'], axis='columns', inplace=True)

# Renomeia a coluna
df_response = df_response.rename(columns = {'id': 'log_id'}, inplace = False)

# Exibe os 5 primeiros registros
df_response.head()

,Content-Length,via,Connection,access-control-allow-credentials,Content-Type,server,access-control-allow-origin,status,size,log_id
0,197,kong/1.3.0,close,true,application/json,nginx,*,500,878,1
1,197,kong/1.3.0,close,true,application/json,nginx,*,204,313,2
2,197,kong/1.3.0,close,true,application/json,nginx,*,500,366,3
3,197,kong/1.3.0,close,true,application/json,nginx,*,401,522,4
4,197,kong/1.3.0,close,true,application/json,nginx,*,401,963,5


In [8]:
# Cria um novo Dataframe com a Rota
df_route = (
    df["route"]
    .apply(pd.Series)
    .merge(df['id'], left_index=True, right_index = True)
)

# Elimina a coluna do Dataframe principal
df.drop(['route'], axis='columns', inplace=True)

# Extrair os dados do header
df_route = (
    df_route["service"]
    .apply(pd.Series)
    .merge(df_route, left_index=True, right_index = True)
)

# Elimina a coluna 
df_route.drop(['service'], axis='columns', inplace=True)

# Renomeia as colunas
df_route = df_route.rename(columns = {'id': 'service_id', 'id_x': 'route_id', 'id_y': 'log_id'}, inplace = False)

# Transforma as colunas com listas em strings
df_route['paths'] = [','.join(map(str, l)) for l in df_route['paths']]
df_route['methods'] = [','.join(map(str, l)) for l in df_route['methods']]
df_route['protocols'] = [','.join(map(str, l)) for l in df_route['protocols']]

# Exibe os 5 primeiros registros
df_route.head()

,service_id,created_at,hosts,route_id,methods,paths,preserve_host,protocols,regex_priority,strip_path,updated_at,log_id
0,c3e86413-648a-3552-90c3-b13491ee07d6,1564823899,miller.com,0636a119-b7ee-3828-ae83-5f7ebbb99831,"GET,POST,PUT,DELETE,PATCH,OPTIONS,HEAD",/,False,"http,https",0,True,1564823899,1
1,d035ffcf-914a-3007-b028-ae18f04d75b4,1561154217,schneider.com,7bedb816-c359-3b8c-a0f9-b03b46c63402,"GET,POST,PUT,DELETE,PATCH,OPTIONS,HEAD",/,False,"http,https",0,True,1561154217,2
2,a5bf08bd-c030-30d5-8009-83a8c30103bf,1545521760,hilpert.com,9bb5b399-23b4-3c5a-b92b-2febdbcfa4e4,"GET,POST,PUT,DELETE,PATCH,OPTIONS,HEAD",/,False,"http,https",0,True,1545521760,3
3,22f8e3a6-01f7-3264-b4b5-9d178df11d06,1543036091,corkery.org,2c31902e-00af-30aa-a04f-08c50ed55785,"GET,POST,PUT,DELETE,PATCH,OPTIONS,HEAD",/,False,"http,https",0,True,1543036091,4
4,22f8e3a6-01f7-3264-b4b5-9d178df11d06,1543036091,corkery.org,2c31902e-00af-30aa-a04f-08c50ed55785,"GET,POST,PUT,DELETE,PATCH,OPTIONS,HEAD",/,False,"http,https",0,True,1543036091,5


In [9]:
# Cria um novo Dataframe com o Service
df_service = (
    df["service"]
    .apply(pd.Series)
    .merge(df['id'], left_index=True, right_index = True)
)

# Elimina a coluna do Dataframe principal
df.drop(['service'], axis='columns', inplace=True)

# Renomeia a coluna
df_service = df_service.rename(columns = {'id_y': 'log_id', 'id_x': 'service_id'}, inplace = False)

# Exibe os 5 primeiros registros
df_service.head()

,connect_timeout,created_at,host,service_id,name,path,port,protocol,read_timeout,retries,updated_at,write_timeout,log_id
0,60000,1563589483,ritchie.com,c3e86413-648a-3552-90c3-b13491ee07d6,ritchie,/,80,http,60000,5,1563589483,60000,1
1,60000,1547309158,terry.org,d035ffcf-914a-3007-b028-ae18f04d75b4,terry,/,80,http,60000,5,1547309158,60000,2
2,60000,1549572086,orn.com,a5bf08bd-c030-30d5-8009-83a8c30103bf,orn,/,80,http,60000,5,1549572086,60000,3
3,60000,1544402652,sauer.com,22f8e3a6-01f7-3264-b4b5-9d178df11d06,sauer,/,80,http,60000,5,1544402652,60000,4
4,60000,1544402652,sauer.com,22f8e3a6-01f7-3264-b4b5-9d178df11d06,sauer,/,80,http,60000,5,1544402652,60000,5


In [10]:
# Cria um novo Dataframe com a latencies
df_latencies = (
    df["latencies"]
    .apply(pd.Series)
    .merge(df['id'], left_index=True, right_index = True)
)

# Elimina a coluna do Dataframe principal
df.drop(['latencies'], axis='columns', inplace=True)

# Renomeia a coluna
df_latencies = df_latencies.rename(columns = {'id': 'log_id'}, inplace = False)

# Exibe os 5 primeiros registros
df_latencies.head()

,proxy,kong,request,log_id
0,1836,8,1058,1
1,1727,10,2185,2
2,896,9,2129,3
3,1209,12,1274,4
4,1708,20,2244,5


In [11]:
# Extrair os dados do authenticated_entity
df = (
    df["authenticated_entity"]
    .apply(pd.Series)
    .merge(df, left_index=True, right_index = True)
)

# Elimina a coluna
df.drop(['authenticated_entity'], axis='columns', inplace=True)

# Extrair os dados do consumer_id
df = (
    df["consumer_id"]
    .apply(pd.Series)
    .merge(df, left_index=True, right_index = True)
)

# Elimina a coluna
df.drop(['consumer_id'], axis='columns', inplace=True)

# Renomeia a coluna
df = df.rename(columns = {'uuid': 'consumer_id'}, inplace = False)

# Exibe as 5 primeiras linhas do dataframe original
df.head()

,consumer_id,upstream_uri,client_ip,started_at,id
0,72b34d31-4c14-3bae-9cc6-516a0939c9d6,/,75.241.168.121,1566660387,1
1,f643db14-a82d-30c2-8c13-889db1d0fcc2,/,88.248.178.118,1556832648,2
2,beceaa24-823b-3bf9-9ae6-c8dada26b264,/,56.54.196.221,1543186574,3
3,7ba24e1f-ed19-31b2-a4be-4114721d63af,/,164.83.216.199,1544521945,4
4,1696f3c4-7732-38f2-9ba8-0eeca8df05d7,/,154.78.181.43,1538597571,5


### Após termos criado todos os dataframes vamos criar uma conexão com o banco de dados MySQL e criar as tabelas do modelo

In [12]:
# Cria a conexão com o banco e inicializa o cursor
cnx = mysql.connector.connect(user='root', password='root',host='127.0.0.1')
cursor = cnx.cursor()
print("Conectado com sucesso.................................")

Conectado com sucesso.................................


In [13]:
# Define o nome do database
DB_NAME = 'melhor_envio'

# Scripts para criar as tabelas
TABLES = {}
TABLES['log'] = (
    "CREATE TABLE `log` ("
    "  `id` int(11) NOT NULL,"
    "  `consumer_id` varchar(50) NOT NULL,"
    "  `upstream_uri` varchar(50) NOT NULL,"
    "  `client_ip` varchar(20) NOT NULL,"
    "  `started_at` varchar(50) NOT NULL,"
    "  PRIMARY KEY (`id`)"
    ") ENGINE=InnoDB")

TABLES['log_request'] = (
    "CREATE TABLE `log_request` ("
    "  `id` int(11) NOT NULL AUTO_INCREMENT,"
    "  `log_id` int(11) NOT NULL,"
    "  `method` varchar(20) NOT NULL,"
    "  `uri` varchar(50) NOT NULL,"
    "  `url` varchar(100) NOT NULL,"
    "  `size` int(11) NOT NULL,"
    "  `querystring` varchar(100) NOT NULL,"
    "  `accept` varchar(50) NOT NULL,"
    "  `host` varchar(100) NOT NULL,"
    "  `user_agent` varchar(50) NOT NULL,"
    "  CONSTRAINT `request_log_fk` FOREIGN KEY (`log_id`) "
    "     REFERENCES `log` (`id`) ON DELETE CASCADE,"
    "  PRIMARY KEY (`id`)"
    ") ENGINE=InnoDB")

TABLES['log_response'] = (
    "CREATE TABLE `log_response` ("
    "  `id` int(11) NOT NULL AUTO_INCREMENT,"
    "  `log_id` int(11) NOT NULL,"
    "  `status` int(11) NOT NULL,"
    "  `size` int(11) NOT NULL,"
    "  `content_length` int(11) NOT NULL,"
    "  `via` varchar(40) NOT NULL,"
    "  `connection` varchar(20) NOT NULL,"
    "  `access_control_allow_credentials` varchar(20) NOT NULL,"
    "  `content_type` varchar(40) NOT NULL,"
    "  `server` varchar(40) NOT NULL,"
    "  `access_control_allow_origin` varchar(40) NOT NULL,"
    "  CONSTRAINT `response_log_fk` FOREIGN KEY (`log_id`) "
    "     REFERENCES `log` (`id`) ON DELETE CASCADE,"
    "  PRIMARY KEY (`id`)"
    ") ENGINE=InnoDB")

TABLES['log_route'] = (
    "CREATE TABLE `log_route` ("
    "  `id` int(11) NOT NULL AUTO_INCREMENT,"
    "  `log_id` int(11) NOT NULL,"
    "  `created_at` varchar(40) NOT NULL,"
    "  `hosts` varchar(100) NOT NULL,"
    "  `route_id` varchar(100) NOT NULL,"
    "  `methods` varchar(100) NOT NULL,"
    "  `paths` varchar(100) NOT NULL,"
    "  `preserve_host` varchar(10) NOT NULL,"
    "  `protocols` varchar(40) NOT NULL,"
    "  `regex_priority` integer(11) NOT NULL,"
    "  `service_id` varchar(100) NOT NULL,"
    "  `strip_path` varchar(40) NOT NULL,"
    "  `updated_at` varchar(40) NOT NULL,"
    "  CONSTRAINT `route_log_fk` FOREIGN KEY (`log_id`) "
    "     REFERENCES `log` (`id`) ON DELETE CASCADE,"
    "  PRIMARY KEY (`id`)"
    ") ENGINE=InnoDB")

TABLES['log_service'] = (
    "CREATE TABLE `log_service` ("
    "  `id` int(11) NOT NULL AUTO_INCREMENT,"
    "  `log_id` int(11) NOT NULL,"
    "  `connect_timeout` integer(11) NOT NULL,"
    "  `created_at` varchar(40) NOT NULL,"
    "  `host` varchar(100) NOT NULL,"
    "  `service_id` varchar(100) NOT NULL,"
    "  `name` varchar(50) NOT NULL,"
    "  `path` varchar(50) NOT NULL,"
    "  `port` varchar(10) NOT NULL,"
    "  `protocol` varchar(40) NOT NULL,"
    "  `read_timeout` integer(11) NOT NULL,"
    "  `retries` integer(11) NOT NULL,"
    "  `updated_at` varchar(40) NOT NULL,"
    "  `write_timeout` integer(11) NOT NULL,"
    "  CONSTRAINT `service_log_fk` FOREIGN KEY (`log_id`) "
    "     REFERENCES `log` (`id`) ON DELETE CASCADE,"
    "  PRIMARY KEY (`id`)"
    ") ENGINE=InnoDB")

TABLES['log_latencies'] = (
    "CREATE TABLE `log_latencies` ("
    "  `id` int(11) NOT NULL AUTO_INCREMENT,"
    "  `log_id` int(11) NOT NULL,"
    "  `proxy` integer(11) NOT NULL,"
    "  `kong` integer(11) NOT NULL,"
    "  `request` integer(11) NOT NULL,"
    "  CONSTRAINT `latencie_log_fk` FOREIGN KEY (`log_id`) "
    "     REFERENCES `log` (`id`) ON DELETE CASCADE,"
    "  PRIMARY KEY (`id`)"
    ") ENGINE=InnoDB")

### Após estabelecer a conexão e definir as tabelas a serem criadas, vamos nos conectar no database desejado e então criar as tabelas caso não existam

In [14]:
# Função para criar o database caso não exista
def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

# Faz a conexão com o database desejado
try:
    cursor.execute("USE {}".format(DB_NAME))
    print("Using database {} ..........".format(DB_NAME))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(DB_NAME))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        print("Database {} created successfully.".format(DB_NAME))
        cnx.database = DB_NAME
    else:
        print(err)
        exit(1)

Database melhor_envio does not exists.
Database melhor_envio created successfully.


In [15]:
# Realiza a criação das tabelas que não existem
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

print("")
print("Tabelas criadas com sucesso!")

Creating table log: OK
Creating table log_request: OK
Creating table log_response: OK
Creating table log_route: OK
Creating table log_service: OK
Creating table log_latencies: OK

Tabelas criadas com sucesso!


### Após todas as tabelas criadas, realizamos o insert dos dados dos logs nas tabelas

In [16]:
# Insere os dados do dataframe 1 por 1

# Tabela LOG
for i,row in df.iterrows():
    sql = "INSERT INTO `log` (`consumer_id`,`upstream_uri`,`client_ip`,`started_at`,`id`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))
    cnx.commit()

In [17]:
# Tabela LOG_REQUEST
for i,row in df_request.iterrows():
    sql = "INSERT INTO `log_request` (`accept`,`host`,`user_agent`,`method`,`uri`, `url`, `size`, `querystring`, `log_id`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))
    cnx.commit()

In [18]:
# Tabela LOG_RESPONSE
for i,row in df_response.iterrows():
    sql = "INSERT INTO `log_response` (`content_length`,`via`,`connection`,`access_control_allow_credentials`,`content_type`,`server`,`access_control_allow_origin`,`status`,`size`,`log_id`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))
    cnx.commit()

In [19]:
# Tabela LOG_ROUTE
for i,row in df_route.iterrows():
    sql = "INSERT INTO `log_route` (`service_id`,`created_at`,`hosts`,`route_id`,`methods`,`paths`,`preserve_host`,`protocols`,`regex_priority`,`strip_path`,`updated_at`,`log_id`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))
    cnx.commit()

In [20]:
# Tabela LOG_SERVICE
for i,row in df_service.iterrows():
    sql = "INSERT INTO `log_service` (`connect_timeout`,`created_at`,`host`,`service_id`,`name`,`path`,`port`,`protocol`,`read_timeout`,`retries`,`updated_at`,`write_timeout`,`log_id`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))
    cnx.commit()

In [21]:
# Tabela LOG_LATENCIES
for i,row in df_latencies.iterrows():
    sql = "INSERT INTO `log_latencies` (`proxy`,`kong`,`request`,`log_id`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))
    cnx.commit()

In [33]:
# Verificar a quantidade de linhas inseridas
for table_name in TABLES:
    cursor.execute("select count(1) from {}".format(table_name))
    row = cursor.fetchone()
    print(f"Tabela {table_name} possue {row[0]} linhas")

Tabela log possue 100000 linhas
Tabela log_request possue 100000 linhas
Tabela log_response possue 100000 linhas
Tabela log_route possue 100000 linhas
Tabela log_service possue 100000 linhas
Tabela log_latencies possue 100000 linhas


### Após todos os inserts, fechamos a conexão com o banco

In [34]:
# Fecha a conexão
cursor.close()
cnx.close()
print("Conexão encerrada com sucesso")

Conexão encerrada com sucesso.........................


### Nesta parte iremos realizar a geração dos relatórios solicitados

In [35]:
try:
    # Cria a conexão com o banco e inicializa o cursor
    cnx = mysql.connector.connect(user='root', password='root',host='127.0.0.1')
    cursor = cnx.cursor()
    print("Conectado com sucesso")
    
    # Definimos o database a ser utilizado
    DB_NAME = 'melhor_envio'

    # Faz a conexão com o database desejado
    cursor.execute("USE {}".format(DB_NAME))
    print("Using database {}".format(DB_NAME))
    
    # Query para o relatório de requisições por consumidor
    sql1 = "select a.consumer_id, a.client_ip, a.started_at, b.method, b.uri, b.url, b.size, \
            b.querystring, b.accept, b.host, b.user_agent from log a \
            inner join log_request b on (a.id = b.log_id) \
            order by a.consumer_id"
    
    print("Gerando relatório de requisições por consumidor.....")
    
    # Cria um dataframe com a query para gerar o relatório
    relatorio1 = pd.read_sql(sql1, con=cnx)
    
    # Exporta o dataframe para um arquivo csv
    relatorio1.to_csv("RelatorioRequisicoesConsumidor.csv", index=False)
    print("Relatório de requisições por consumidor gerado com sucesso! -> RelatorioRequisicoesConsumidor.csv")
    
    # Query para o relatório de requisições por serviço
    sql2 = "select a.connect_timeout, a.created_at, a.host, a.service_id, a.name, a.path, a.port, \
            a.protocol, a.read_timeout, a.retries, a.updated_at, a.write_timeout, b.method, b.uri, \
            b.url, b.size, b.querystring, b.accept, b.host, b.user_agent from log_service a \
            inner join log_request b on (a.log_id = b.log_id) \
            order by a.service_id"
    
    print("Gerando relatório de requisições por serviço.....")
    
    # Cria um dataframe com a query para gerar o relatório
    relatorio2 = pd.read_sql(sql2, con=cnx)
    
    # Exporta o dataframe para um arquivo csv
    relatorio2.to_csv("RelatorioRequisicoesServico.csv", index=False)
    print("Relatório de requisições por serviço gerado com sucesso! -> RelatorioRequisicoesServico.csv")
    
    # Query para o relatório tempo médio de request , proxy e kong por serviço
    sql3 = "select avg(b.proxy) as tempo_medio_proxy, avg(b.kong) as tempo_medio_kong, \
            avg(b.request) as tempo_medio_request, a.service_id from log_service a \
            inner join log_latencies b on (a.log_id = b.log_id) \
            group by a.service_id order by a.service_id"
    
    print("Gerando relatório de requisições por serviço.....")
    
    # Cria um dataframe com a query para gerar o relatório
    relatorio3 = pd.read_sql(sql3, con=cnx)
    
    # Exporta o dataframe para um arquivo csv
    relatorio3.to_csv("RelatorioLatenciesServico.csv", index=False)
    print("Relatório de tempo médio de request, proxy e kong por serviço gerado com sucesso! -> RelatorioLatenciesServico.csv")
    
except mysql.connector.Error as err:
    print("Erro : ",err)
finally:
    cursor.close()
    cnx.close()
    print("Conexão encerrada com sucesso")

Conectado com sucesso
Using database melhor_envio
Gerando relatório de requisições por consumidor.....
Relatório de requisições por consumidor gerado com sucesso! -> RelatorioRequisicoesConsumidor.csv
Gerando relatório de requisições por serviço.....
Relatório de requisições por serviço gerado com sucesso! -> RelatorioRequisicoesServico.csv
Gerando relatório de requisições por serviço.....
Relatório de tempo médio de request, proxy e kong por serviço gerado com sucesso! -> RelatorioLatenciesServico.csv
Conexão encerrada com sucesso
